In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from cellpose import plot, utils, io
import cellpose
from scipy.ndimage import measurements
import seaborn as sns
from scipy.spatial import Voronoi, voronoi_plot_2d, Delaunay, ConvexHull
from scipy.ndimage import center_of_mass
import matplotlib.cm as cm
import matplotlib
from itertools import *
from pylab import *
import time
from collections import defaultdict, Counter
import itertools
from itertools import combinations
from shapely.geometry import LineString, MultiPolygon, MultiPoint, Point
import collections
from matplotlib.collections import LineCollection
from collections import OrderedDict
from scipy.ndimage import label
import matplotlib.colors as mcolors
import matplotlib.patches as patches
import networkx as nwx
import math
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from scipy.spatial.distance import pdist, squareform
from pathlib import Path
from PIL import Image
import pandas as pd
from itertools import chain

matplotlib.rcParams.update({'font.size': 9})
matplotlib.rcParams['mathtext.fontset'] = 'dejavusans'


In [ ]:
def find_neighbors(labeled_matrix):
    neighbors = {label: set() for label in unique_labels if label != 0}

    rows, cols = labeled_matrix.shape
    for i in range(rows):
        for j in range(cols):
            current_label = labeled_matrix[i, j]
            if current_label == 0:
                continue

            # Check 4-connected neighbors (up, down, left, right)
            for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                ni, nj = i + di, j + dj
                if 0 <= ni < rows and 0 <= nj < cols:
                    neighbor_label = labeled_matrix[ni, nj]
                    if neighbor_label != 0 and neighbor_label != current_label:
                        neighbors[current_label].add(neighbor_label)

    return neighbors

def relabel_dataframe(labeled_image, dataframe):
    # Initialize a new column for the new labels
    new_labels = []

    for index, row in dataframe.iterrows():
        # Get the x and y positions of the centroid (assuming integers)
        x, y = int(row['Center of the object_0']), int(row['Center of the object_1'])
        
        # Ensure x and y are within bounds of the labeled_image
        if 0 <= x < labeled_image.shape[1] and 0 <= y < labeled_image.shape[0]:
            # Get the label at the centroid position from labeled_image
            label = labeled_image[y, x]
        else:
            label = -1  # Assign a default or error label if out of bounds
        
        # Append the label to the new list
        new_labels.append(label)
    
    # Add the new labels as a column to the dataframe
    dataframe['new_label'] = new_labels

    return dataframe

def compute_morans_I(neighbor_dict, dark_cells):

    all_cells = set(neighbor_dict.keys()) | {n for neighbors in neighbor_dict.values() for n in neighbors}

    # Assign values: dark cells = 1, light cells = 0
    cell_values = {cell: 1 if cell in dark_cells else 0 for cell in all_cells}

    # Compute mean cell value
    x_values = np.array([cell_values[cell] for cell in all_cells])
    x_mean = np.mean(x_values)

    # Compute denominator (variance term)
    denominator = np.sum((x_values - x_mean) ** 2)
    if denominator == 0:
        return np.nan  # Avoid division by zero

    # Compute numerator and total weight sum W
    numerator = 0
    W = sum(len(neighbors) for neighbors in neighbor_dict.values())

    for i in all_cells:
        neighbors = neighbor_dict.get(i, set())  # Get neighbors or empty set if none
        for j in neighbors:
            if j in all_cells:
                wij = 1  # Weight of neighbor connection
                numerator += wij * (cell_values[i] - x_mean) * (cell_values[j] - x_mean)

    # Compute Moran's I
    moran_I = (len(all_cells) / W) * (numerator / denominator) if W > 0 else np.nan
    return moran_I

data = {}
images = {}
SIZES = {}
DL = {}
LL = {}
Ncells = {}
Ndark_cells = {}
density_dark = {}
ND = {}
LSP = {}
CSD = {}
CS = {}
NEIGHBORS = {}
ANGLES = {}
ORDER = {}
MORAN = {}
DARK_CENTROIDS = {}


folders = ['thallus/sections']
folder_paths = [Path(f) for f in folders]
all_npy_paths = chain.from_iterable(folder.glob('*_seg.npy') for folder in folder_paths)

k = -1;

for npy_path in all_npy_paths:
#for npy_path in all_npy_paths.glob('*_seg.npy'):
    
    k = k + 1

    plt.figure(1,figsize = (10,10))
    
    img_path = npy_path.with_name(npy_path.stem[:-4] + '.jpg')
    csv_path = npy_path.with_name(npy_path.stem[:-4] + '_table.csv')

    if not csv_path.exists():
        print(f"Skipping {csv_path}, as it does not exist.")
        continue

    dat = np.load(npy_path, allow_pickle=True).item()
    dat_csv = pd.read_csv(csv_path, index_col = None)

    if img_path.exists():
        img = io.imread(img_path)
        print(f"Loaded {npy_path.name} and its associated image {img_path.name}")
    else:
        print(f"Warning: No associated image found for {npy_path.name}")

    size_threshold = 10

    data[k] = dat
    images[k] = img

    labeled_image = data[k]['masks']
    unique_labels = np.unique(labeled_image)

    dat_csv = relabel_dataframe(labeled_image, dat_csv)
    centroidss = {idx: (row['Center of the object_1'], row['Center of the object_0']) for idx, row in dat_csv.iterrows() }

    dark_labels = np.array(dat_csv[(dat_csv['Predicted Class'].isin(['Oil body'])) & (dat_csv['Size in pixels'] > size_threshold)]['new_label'])
    light_labels = np.array(dat_csv[(dat_csv['Predicted Class'].isin(['Epidermal'])) & (dat_csv['Size in pixels'] > size_threshold)]['new_label'])
    ap_labels = np.array(dat_csv[(dat_csv['Predicted Class'].isin(['Air pore'])) & (dat_csv['Size in pixels'] > size_threshold)]['new_label'])

    all_labels = np.array(dat_csv[(dat_csv['Size in pixels'] > size_threshold)]['new_label']) 
    #all_labels = np.array([int(k) for k in range(1,len(all_labels)+1)])

    dat_csv = dat_csv[dat_csv['new_label'].isin(all_labels)]
    centroidss = {idx: (row['Center of the object_1'], row['Center of the object_0']) for idx, row in dat_csv.iterrows() }

    # plot image with masks overlaid
    #mask_RGB = plot.mask_overlay(images[k], dat['masks'],
    #                        colors=np.array(dat['colors']))

    # plot image with outlines overlaid in red
    outlines = utils.outlines_list(data[k]['masks'])

    cmap = matplotlib.colors.ListedColormap(np.random.rand( 256,3))

    plt.imshow(img, alpha = 1)
    #plt.imshow(data[k]['masks'], alpha = 0.5, cmap = cmap)

    #for o in outlines:
        #plt.plot(o[:,0], o[:,1], color='red', linewidth = 1.)
        
    points = []
    centroids = {}
    sizes = {}
    
    for label in unique_labels:
        if label == 0:
            continue  # Skip background or unlabeled areas
        mask = labeled_image == label
        centroid = center_of_mass(mask)
        centroids[label] = centroid
        size = sum(mask)
        sizes[label] = size

        # Calculate the average RGB values for the masked region
    #    avg_color = np.mean(img[mask], axis=0)

        # Determine if the cell is dark (average RGB values close to black)
    #    if np.all(avg_color <= threshold):
    #        dark_labels.append(label)
    #    else:
    #        light_labels.append(label)

    #SIZES[k] = dat_csv['Size in pixels'].values
    SIZES[k] = sizes

    DL[k] = dark_labels    
    LL[k] = light_labels    

    dark_centroids = [centroids[j] for j in dark_labels if j in centroids]
    DC = np.array(dark_centroids)
        
    #[plt.text(DC[:,1][i], DC[:,0][i], str(j), color='w', fontsize=7, ha='center', va='center') for i, j in enumerate(dark_labels)]
    
    Ncells[k] = len(dat_csv)
    Ndark_cells[k] = len(dark_labels)
    density_dark[k] = Ndark_cells[k]/Ncells[k]
        
    #plt.axis([2215,2382,1700,1800])

    #0.1mm = 165pixels ---> 1mm = 1650pixels ---> 1pixel = 1/1650mm
    
    dark_cells = dark_labels
    
    neighbors = collections.OrderedDict(find_neighbors(labeled_image))
    #neighbors = collections.OrderedDict((new_key, value) for new_key, (old_key, value) in enumerate(neighbors.items()))
    
    dark_cell_neighbors = defaultdict(set)
    
    for cell in dark_cells:
        if cell in neighbors:
            dark_cell_neighbors[cell] = neighbors[cell].intersection(dark_cells)
    
    # If you want to convert it to a regular dictionary (optional)
    dark_cell_neighbors = dict(dark_cell_neighbors)
    
    # Function to perform DFS and find connected components
    def dfs(cell, visited, component):
        visited.add(cell)
        component.add(cell)
        for neighbor in dark_cell_neighbors[cell]:
            if neighbor not in visited:
                dfs(neighbor, visited, component)
    
    # Find all clusters
    visited = set()
    clusters = []
    
    for cell in dark_cells:
        if cell not in visited:
            component = set()
            dfs(cell, visited, component)
            clusters.append(component)

    # Calculate the sizes of clusters
    cluster_sizes = [len(cluster) for cluster in clusters]
    cluster_sizes = Counter(cluster_sizes)
    
    def compute_gabriel_graph(points):
        # Compute the Delaunay triangulation
        delaunay = Delaunay(points)
        edges = []
    
        # Iterate through the edges of the Delaunay triangulation
        for simplex in delaunay.simplices:
            for i in range(3):
                for j in range(i + 1, 3):
                    p1, p2 = points[simplex[i]], points[simplex[j]]
                    mid_point = (p1 + p2) / 2
                    radius = np.linalg.norm(p1 - p2) / 2  # Correct radius calculation (half the distance)
    
                    is_gabriel = True
    
                    # Check if any other point lies within the circle
                    for k in range(len(points)):
                        if k != simplex[i] and k != simplex[j]:  # Exclude points on the current edge
                            if np.linalg.norm(points[k] - mid_point) < radius:
                                is_gabriel = False
                                break
    
                    if is_gabriel:
                        edges.append((simplex[i], simplex[j]))
    
        return edges

    ################
    
    points = np.array(list(centroids.values()))
    
    dark_points = DC

    gabriel_dark = compute_gabriel_graph(dark_points)
    
    G_dark = nwx.Graph()
    for point in dark_points:
        G_dark.add_node(tuple(point))
    G_dark.add_edges_from(gabriel_dark)
    
    G = nwx.Graph()
    
    for region, neighbors_set in neighbors.items():
        G.add_node(region)  # Add the region as a node
        for neighbor in neighbors_set:
            G.add_edge(region, neighbor)  # Add an edge between the region and its neighbors
    
    def find_simplices_from_gabriel_graph(G):
        cliques = list(nwx.find_cliques(G))
        # Filter cliques to find simplices (size 3 or more)
        simplices = [clique for clique in cliques if len(clique) >= 3]
        return simplices
        
    def find_all_cliques(G):
        # Find all cliques in the graph
        cliques = list(nwx.find_cliques(G))
        return cliques

    #pos = {region: (x, y) for region, (y,x) in centroids.items()}
    #nwx.draw_networkx(G, pos, node_color = 'k', edge_color = 'k', node_size = 15, width = 1.5, with_labels=False, alpha = 1)

    #plt.plot(DC[:,1], DC[:,0], 'C0o', markersize = 7)  # centroids are plotted in red

    #gabriel_dark_simplices = np.array(find_simplices_from_gabriel_graph(G_dark))
    ###making np.array sometimes gives errors because it finds simplices of 4 elements instead of 3, and cannot convert to array
    gabriel_dark_cliques = find_all_cliques(G_dark)
    
    gabriel_neighbors = {label: [] for label in dark_labels}
    angles = []
    
    # Find neighbors from the Gabriel graph
    for clique in gabriel_dark_cliques:
        for i in range(len(clique)):
            for j in range(len(clique)):
                if i != j:
                    p1 = dark_labels[clique[i]]
                    p2 = dark_labels[clique[j]]
                    gabriel_neighbors[p1].append(p2)

        if len(clique) == 3:  # Ensure it's a triangle (this I have to check for the angles, as sometimes they are not triangles)
            ppp = [centroids[dark_labels[point]] for point in clique]
            
            distances = squareform(pdist(ppp))
            
            a, b, c = distances[0, 1], distances[1, 2], distances[0, 2]
            
            angle_A = np.arccos((b**2 + c**2 - a**2) / (2 * b * c))
            angle_B = np.arccos((a**2 + c**2 - b**2) / (2 * a * c))
            angle_C = np.arccos((a**2 + b**2 - c**2) / (2 * a * b))
            
            angles.extend([angle_A, angle_B, angle_C])

    all_gabriel_angles = np.degrees(angles)
    
    # Remove duplicates in neighbors
    for key in gabriel_neighbors.keys():
        gabriel_neighbors[key] = list(set(gabriel_neighbors[key]))
    
    def get_neighbors(label):
        if label in gabriel_neighbors:
            return gabriel_neighbors[label]
        else:
            return []
    
    dark_neighbors = {label: get_neighbors(label) for label in dark_labels}
    
    #######
    ###to compute neighbour density
    def compute_avg_dark_neighbors(clusters, dark_cells, neiList):
        dark_cells_set = set(dark_cells)  # Convert dark_cells to a set for fast lookup
        cs = []
        nd = []
    
        for cluster in clusters:
            dark_neighbor_counts = []
    
            # Check if the cluster is a set or just an int
            if isinstance(cluster, int):
                cluster = {cluster}  # Convert single int to a set
    
            for cell in cluster:
                NN = neiList.get(cell, set())  # Safely get neighbors
                dark_neighbors = sum(1 for nei in NN if nei in dark_cells_set)
                dark_neighbor_counts.append(dark_neighbors)
    
            avg_dark_neighbors = np.mean(dark_neighbor_counts) if dark_neighbor_counts else 0
            cs.append(len(cluster))
            nd.append(avg_dark_neighbors)
    
        return cs, nd
    
    cs, nd = compute_avg_dark_neighbors(clusters, dark_cells, neighbors)
        
    # Function to find the shortest path between two points
    def find_shortest_path(G, start_point, end_point):
        shortest_path = nwx.shortest_path(G, source=start_point, target=end_point, weight='weight')
        return shortest_path
    
    all_pairs = set()
    
    for label in dark_labels:
        for nei in dark_neighbors[label]:
            pair = tuple(sorted((label, nei)))
            all_pairs.add(pair)
                    
    len_shortest_path = np.zeros(len(all_pairs));

    all_pairs = list(all_pairs)

    for edge in gabriel_dark:
        p1, p2 = edge
        point1 = dark_points[p1]
        point2 = dark_points[p2]
        plt.plot([point1[1], point2[1]], [point1[0], point2[0]], 'C0o-', linewidth=2, marker='.', markersize = 5)
     
    kk = -1
    angles = []

    for l,j in all_pairs:
        
        kk = kk + 1
    
        start_label = l
        end_label = j
    
        shortest_path = find_shortest_path(G, start_label, end_label)
    
        path_points = [centroids[k] for k in shortest_path]
        path_points = np.asarray([list(elem) for elem in path_points])

        plt.plot(path_points[:, 1], path_points[:, 0], color='r', linewidth=1.5, marker='.', markersize = 5, alpha = 1)

        len_shortest_path[kk] = len(shortest_path)-1

    cell_values = {cell: 1 for cell in dark_labels}
    cell_values.update({cell: 0 for cell in light_labels})

    missing_keys = {n for nn in neighbors.values() for n in nn} - set(cell_values.keys())

    order_parameter = 0
    for cell, nn in neighbors.items():
        if cell not in cell_values:
            continue  # Skip if the cell itself is missing
    
        valid_neighbors = [n for n in nn if n in cell_values]  # Only keep valid neighbors
        if valid_neighbors:  # Ensure we have at least one valid neighbor
            avg_diff = sum(abs(cell_values[cell] - cell_values[n]) for n in valid_neighbors) / len(valid_neighbors)
            order_parameter += avg_diff
    
    order_parameter = order_parameter/len(cell_values)
    moran_I = compute_morans_I(neighbors, dark_labels)
        
    ANGLES[k] = all_gabriel_angles
    CS[k] = cs 
    CSD[k] = cluster_sizes 
    NEIGHBORS[k] = neighbors
    ND[k] = nd
    LSP[k] = len_shortest_path
    ORDER[k] = order_parameter
    MORAN[k] = moran_I
    DARK_CENTROIDS[k] = DC

    print(Ncells[k],Ndark_cells[k],density_dark[k])
    #plt.savefig('image_network.pdf', transparent = True, bbox_inches='tight' )

    plt.show()

    plt.show()


In [ ]:
combined_angles = [value for array in ANGLES.values() for value in array]
combined_angles =  np.radians(combined_angles)

print(density_dark)
print(Ncells)

print('Average number of oil body cells:',np.mean(list(Ndark_cells.values())),'±',np.std(list(Ndark_cells.values())))

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize = (3,3))

num_bins = 30

counts, bins, patches = ax.hist(combined_angles, bins=num_bins, density=True, edgecolor='black', linewidth = 0.8)

norm_angles = (bins[:-1] + bins[1:]) / 2  # Midpoint of each bin
norm_angles = np.degrees(norm_angles)  # Convert to degrees for the colormap
norm_angles = norm_angles / 120.0  # Normalize to [0, 1] for the colormap

cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["white","burlywood","saddlebrown","saddlebrown"])

for angle, patch in zip(norm_angles, patches):
    color = cmap(angle)
    patch.set_facecolor(color)

ax.set_theta_direction(1)  # Clockwise direction
ax.set_theta_offset(0)  # Start at 0 degrees on the right

ax.set_thetamin(0); ax.set_thetamax(120)

ax.set_xticks([0,np.pi/6,np.pi/3,np.pi/2,2*np.pi/3])
ax.set_yticks([])


ax.grid(linewidth = 1, alpha = 0.4)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=120))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', label='angle', shrink = 0.9, ticks=[0,40,80,120], pad = -0.05)

plt.savefig('histogram_angles.pdf', transparent = True, bbox_inches='tight' )
plt.savefig('histogram_angles.svg')

plt.show()


#############


fig, (ax0,ax1,ax2,ax3) = plt.subplots(1, 4, figsize=(13,2), width_ratios=[0.7,2,2,2])
fig.subplots_adjust(wspace=0.35)

ddd = [list(density_dark.values()),list(density_dark.values())]

sns.boxplot(ddd, widths = 0.5, palette = ['y','y'], ax = ax0, linewidth = 1.2)
sns.stripplot(data=ddd, ax=ax0, marker = 'o', color='w', edgecolor = 'k', jitter = True, linewidth=0.7,size=2.3, alpha = 0.7)

ax0.spines[['right', 'top']].set_visible(False)

ax0.set_ylim(0.,0.06)
ax0.set_xlim(-0.6,0.6)

ax0.set_ylabel(r'dark cell density $\rho$')
ax0.set_xticklabels(['exp.'])


########################

density_av = np.mean(list(density_dark.values()))
density_std = np.std(list(density_dark.values()))

lsp_together = np.concatenate(list(LSP.values()))
csd_together = defaultdict(int)

lspt = collections.OrderedDict(sorted(Counter(lsp_together).items()))

for sub_dict in CSD.values():
    for key, value in sub_dict.items():
        csd_together[key] += value

csd_together = dict(collections.OrderedDict(sorted(csd_together.items())))

all_bins = set()
for values in LSP.values():
    all_bins.update(values)

all_bins = sorted(all_bins)

histograms = []

for values in LSP.values():
    histogram = Counter(values)
    histogram_values = list(histogram.values())
    total = sum(histogram_values)
    
    #print("Histogram values:", histogram_values)
    #print("Total:", total)
    
    if total > 0:
        histogram_normalized = {k: v / total for k, v in histogram.items()}
    else:
        histogram_normalized = {}
    
    histogram_values = [histogram_normalized.get(bin_key, 0) for bin_key in all_bins]
    histograms.append(histogram_values)

histograms_array = np.array(histograms)

mean_histogram = np.mean(histograms_array, axis=0)
std_histogram = np.std(histograms_array, axis=0)

ax1.bar(all_bins, mean_histogram, yerr=std_histogram, capsize=3, color='y', alpha=0.8, width=1, edgecolor='k', align='center')
#ax1.plot(all_bins, mean_histogram, 'wo', alpha=0.8, ms=4, markeredgecolor='k')

ax1.set_xlabel('cells apart')
ax1.set_ylabel('frequency')
#ax1.grid(alpha = 0.2, linewidth = 1.);
ax1.set_xticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]);
ax1.set_xticklabels(['0', '1', '2', '3', '4', '5', '6', '7','8', '9', '10', '', '', '', '', '', '', '','','20'])

ax1.set_xlim(0.5,11.5)
ax1.set_ylim(0,1)
#ax1.legend(loc = 'upper right', fontsize = 12)

ax1.spines[['right', 'top']].set_visible(False)

#ax1.annotate(r'$\langle \rho \rangle = %.2f \pm %.2f$'%(density_av,density_std),[5.5,0.2], fontsize = 10)

# Add inset for slope
inset_ax = ax1.inset_axes([0.51, 0.5, 0.47, 0.47])

#inset_ax.plot(all_bins, mean_histogram, 'wo', alpha=0.8, ms=4, markeredgecolor='k')
inset_ax.errorbar(all_bins, mean_histogram, yerr=std_histogram, capsize=2, fmt='o-', ecolor='k', linewidth=1, elinewidth=1, 
                  ms=3, color = 'y', markeredgecolor='k', markeredgewidth=0.7)

###this is to plot all the data together, without averaging and errors
#inset_ax.plot(list(lspt.keys()), list(lspt.values())/sum(list(lspt.values())), 'go-')


inset_ax.set_yscale('log')
inset_ax.set_xscale('log')
inset_ax.grid(alpha = 0.2, linewidth = 1.);
inset_ax.set_xticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]);
inset_ax.set_xticklabels(['1', '', '', '', '5', '', '', '','', '10', '', '', '', '', '', '', '', '','','20'], fontsize = 9)
inset_ax.set_yticks([0.000001,0.00001,0.0001,0.001,0.01,0.1,1,2]);
inset_ax.set_yticklabels(['$10^{-6}$', '','', '$10^{-3}$', '', '', '$10^{0}$', ''], fontsize = 9)


csd = list(csd_together.values())/sum(list(csd_together.values()))
cskey = list(csd_together.keys())

ax2.bar(cskey, csd,  color='y', alpha=0.8, width=1, edgecolor='k', align='center')
#ax2.plot(cskey, csd, 'wo-', alpha=0.8, ms=4, markeredgecolor='k')

ax2.set_xlabel('cluster size')
ax2.set_ylabel('frequency')
#ax2.grid(alpha = 0.2, linewidth = 1.);
ax2.set_xticks([1,2,3,4,5,6,7,8,9,10]);
ax2.set_xticklabels(['1', '2', '3', '4', '5', '6', '7','8', '9','10'])

ax2.set_xlim(0.5,10.5)
ax2.set_ylim(0,1)
#ax2.legend(loc = 'upper right', fontsize = 12)

ax2.spines[['right', 'top']].set_visible(False)

# Add inset for slope
inset_ax = ax2.inset_axes([0.45, 0.5, 0.4, 0.47])

#inset_ax.plot(all_bins, mean_histogram, 'wo', alpha=0.8, ms=4, markeredgecolor='k')
inset_ax.errorbar(cskey, csd, capsize=2, fmt='o-', ecolor='k', linewidth=1, elinewidth=1, 
                  ms=3, color = 'y', markeredgecolor='k', markeredgewidth=0.7)
#inset_ax.plot(all_bins, mean_histogram, 'k-', alpha=0.8, ms=4, markeredgecolor='k')

inset_ax.set_xlim([1,20])

inset_ax.set_yscale('log')
inset_ax.set_xscale('log')
inset_ax.grid(alpha = 0.2, linewidth = 1.);
inset_ax.set_xticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]);
inset_ax.set_xticklabels(['1', '', '', '', '', '', '', '','', '10','','','','','','','','','','20'], fontsize = 9)
inset_ax.set_yticks([0.000001,0.00001,0.0001,0.001,0.01,0.1,1,2]);
inset_ax.set_yticklabels(['$10^{-6}$','','', '$10^{-3}$', '', '', '$10^{0}$', ''], fontsize = 9)


######################################

size_density_data = defaultdict(list)

for sim in ND.keys():
    cluster_sizes = CS[sim]
    densities = ND[sim]

    for size, density in zip(cluster_sizes, densities):
        if size == 1:
            size_density_data[size].append(0.0)
        else:
            size_density_data[size].append(density)
            

for sim in CS.keys():
    for size, density in zip(CS[sim], ND[sim]):
        size_density_data[size].append(density)

cluster_sizes = np.array(sorted(size_density_data.keys()))
mean_densities = np.array([np.mean(size_density_data[size]) for size in cluster_sizes])
std_densities = np.array([np.std(size_density_data[size]) for size in cluster_sizes])

xx = np.linspace(2,20,100)
yy = 2 - 2/xx

ax3.errorbar(cluster_sizes, mean_densities, yerr=std_densities, fmt='-o', ms = 5, 
             ecolor='k', color = 'y', markeredgecolor = 'k', capsize = 3, alpha = 0.7)
ax3.plot(xx,yy,'k--', alpha = 0.5, label = r'$2-2/k$')

ax3.fill_between(cluster_sizes, mean_densities  + std_densities, mean_densities  - std_densities, color = "y",
                 alpha = 0.3, linestyle = '--', linewidth = 2)

ax3.set_xlabel('cluster size')
ax3.set_ylabel('neighbour density')
#ax3.grid(alpha = 0.2, linewidth = 1.);
ax3.set_xticks([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]);
ax3.set_xticklabels(['','', '2', '', '4', '', '6', '', '8','', '10','', '12','','14',''])

ax3.set_xlim(2,14.5)
ax3.set_ylim(1,3)
#ax3.legend(loc = 'upper right', fontsize = 12)

ax3.spines[['right', 'top']].set_visible(False)

#########

sizes_of_interest = [3, 4, 5]

# Mapping of neighbor densities to labels (I, II, III, IV)
density_to_label = {
    1.33: 'I',
    1.50: 'I',
    1.60: 'I',
    2.00: 'II',
    2.40: 'III',
    2.50: 'III',
    2.80: 'IV',
}

# # Normalize the color map to the maximum number of labels
# labels = ['1', '2', '3', '4']
# cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["silver","lightseagreen"])
# colors = cmap(np.linspace(0, 1, len(labels)))
# label_to_color = dict(zip(labels, colors))

# TC = {}
# ss = -1
# for size in sizes_of_interest:
#     ss = ss + 1
#     densities = size_density_data[size]
    
#     # Count occurrences of each unique density
#     density_counts = Counter(densities)

#     densities = sorted(set(density_counts.keys()))
#     density_to_label = {density: f'{i+1}' for i, density in enumerate(densities)}
#     transformed_counter = Counter({density_to_label[density]: count for density, count in density_counts.items()})

#     # Prepare labels and sizes for the pie chart
#     pie_labels = list(transformed_counter.keys())
#     sizes = list(density_counts.values())

#     TC[ss] = transformed_counter


# for i, size in enumerate(sizes_of_interest):
#     transformed_counter = TC[i]
    
#     # Extract and sort the sizes and corresponding labels
#     pie_labels = list(transformed_counter.keys())
#     sizes = [transformed_counter[label] for label in pie_labels]

#     # Sort labels and sizes based on sizes in descending order
#     sorted_indices = np.argsort(sizes)[::-1]
#     sorted_labels = [pie_labels[i] for i in sorted_indices]
#     sorted_sizes = [sizes[i] for i in sorted_indices]
#     sorted_colors = [label_to_color[label] for label in sorted_labels]
    
#     # Create an inset axes at the x position corresponding to the cluster size
#     inset_ax = ax3.inset_axes([size/4.4 - 0.68, 0.65, 0.3, 0.3])  # Adjust position and size
#     wedges, texts = inset_ax.pie(sorted_sizes, startangle=90, colors=sorted_colors, 
#                                  wedgeprops={"edgecolor": "black", 'linewidth': 1, 'antialiased': True})
    
#     inset_ax.axis('equal')
#     inset_ax.set_title(f'$k={size}$', y=0.9, fontsize=9)

# # Add a legend with labels corresponding to the types
# handles = [plt.Line2D([0], [0], color=label_to_color[label], lw=4) for label in labels]
# inset_ax.legend(bbox_to_anchor=(0.9, 0.65, 0.5, 0.5), labels=['I','II','III','IV'], fontsize = 8)

plt.savefig('histogram_cell_distance.pdf', transparent = True, bbox_inches='tight' )
plt.savefig('histogram_cell_distance.svg')

plt.show()


In [ ]:
from scipy.spatial import ConvexHull

prop_1cluster = [values.count(1) / len(values) for values in CS.values()]

print(len(prop_1cluster))
print(list(MORAN.values()))

fig,(ax1,ax2) = plt.subplots(1,2,figsize = (6.,2.))
fig.subplots_adjust(wspace=0.35)

ax1.spines[['right', 'top']].set_visible(False)
ax2.spines[['right', 'top']].set_visible(False)

#ax.scatter(list(DENSITY_DARK.values()),prop_1cluster)

#ax.scatter(list(MORAN.values()),list(ORDER.values()))

#ax.scatter(list(DENSITY_DARK.values()),list(MORAN.values()))

x = np.array(list(MORAN.values()))
y = np.array(prop_1cluster)

points = np.column_stack((x, y))
hull = ConvexHull(points)

hull_points = points[hull.vertices]

ax1.boxplot(list(MORAN.values()), positions=[0.5], widths = 0.05, vert = False, showfliers = False)
ax1.boxplot(prop_1cluster, positions=[0.15], widths = 0.015, showfliers = False)

ax1.fill(hull_points[:, 0], hull_points[:, 1], color='y', alpha=0.5, linewidth = 0)
ax1.scatter(x, y, color='y', s=18, alpha=0.5, linewidth = 0.5)

ax1.set_xlim(-0.05,0.05)
ax1.set_ylim(0.6,1.05)

ax1.set_yticks([0.6,0.8,1])
ax1.set_yticklabels([0.6,0.8,1])

ax1.set_xticks([-0.05,0,0.05])
ax1.set_xticklabels([-0.05,0,0.05])

ax1.set_xlabel("Moran's $I$")
ax1.set_ylabel("idioblast proportion")

# plt.scatter(prop_1cluster,list(ORDER.values()))

#plt.hist(density_dark.values(), bins = 10)


WL_mean = np.array([np.mean(i) for i in LSP.values()])
WL_std = np.array([np.std(i) for i in LSP.values()])

WL_CV = WL_std/WL_mean

points = np.column_stack((WL_mean, WL_CV))
hull = ConvexHull(points)

hull_points = points[hull.vertices]

ax2.boxplot(WL_mean, positions=[0.3], widths = 0.07, vert = False, showfliers = False)
ax2.boxplot(WL_CV, positions=[6.7], widths = 0.4, showfliers = False)

ax2.fill(hull_points[:, 0], hull_points[:, 1], color='y', alpha=0.5, linewidth = 0)
ax2.scatter(WL_mean,WL_CV, color='y', s=18, alpha=0.5, linewidth = 0.5)

ax2.set_xlim(3,7)
ax2.set_ylim(0.1,0.7)

ax2.set_yticks([0.1,0.2,0.3,0.4,0.5,0.6,0.7])
ax2.set_yticklabels([0.1,0.2,0.3,0.4,0.5,0.6,0.7])

ax2.set_xticks([3,4,5,6,7])
ax2.set_xticklabels([3,4,5,6,7])

ax2.set_xlabel("average oil body cell distances $\lambda$")
ax2.set_ylabel("CV$_\lambda$")

plt.savefig('scatter_properties_marchantia_exp.pdf', transparent = True, bbox_inches='tight' )


print(np.mean(list(Ncells.values())))

In [ ]:
import csv

np.save("exp_marchantia_density.npy", list(density_dark.values()))
np.save("exp_marchantia_moran.npy", list(MORAN.values()))

with open("CS_marchantia_exp.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Key", "Value"])  # Header
    for k, v in CS.items():
        writer.writerow([k, ",".join(map(str, v))])  # Convert the list to a comma-separated string

with open("ND_marchantia_exp.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Key", "Values"])  # Header
    for key, values in ND.items():
        writer.writerow([key] + values)

with open("LSP_marchantia_exp.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Key", "Array"])  # Header
    for k, v in LSP.items():
        writer.writerow([k, ",".join(map(str, v))])  # Convert array to comma-separated string

with open("CSD_marchantia_exp.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Key", "InnerKey", "Value"])  # Header
    for outer_key, inner_dict in CSD.items():
        for inner_key, value in inner_dict.items():
            writer.writerow([outer_key, inner_key, value])